# Netcore AI Marketing Suite - Interactive Demo

This notebook demonstrates the capabilities of the Marketing Chatbot and allows you to interactively test different features.

In [1]:
import sys
import os
sys.path.append('..')

from quick_wins.marketing_chatbot.chatbot import MarketingChatbot
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## Initialize the Chatbot

First, let's initialize the marketing chatbot. You can choose between a lighter model for faster response or a more powerful model for better quality.

In [2]:
model_dropdown = widgets.Dropdown(
    options=['distilgpt2', 'facebook/opt-125m', 'mistralai/Mistral-7B-Instruct-v0.1'],
    value='distilgpt2',
    description='Model:',
)

quantize_checkbox = widgets.Checkbox(
    value=True,
    description='Enable quantization',
    disabled=False
)

initialize_button = widgets.Button(description="Initialize Chatbot")
output = widgets.Output()

display(model_dropdown, quantize_checkbox, initialize_button, output)

def on_initialize_button_clicked(b):
    with output:
        clear_output()
        print(f"Initializing chatbot with {model_dropdown.value} (quantized={quantize_checkbox.value})...")
        global chatbot
        chatbot = MarketingChatbot(model_name=model_dropdown.value, quantize=quantize_checkbox.value)
        print("Chatbot initialized! You can now use it in the sections below.")

initialize_button.on_click(on_initialize_button_clicked)

## 1. Marketing Chatbot Interaction

Ask marketing-related questions to the AI chatbot.

In [3]:
question_input = widgets.Text(
    value='How can I improve my email open rates?',
    placeholder='Type your marketing question here',
    description='Question:',
    disabled=False,
    style={'description_width': 'initial'}
)

ask_button = widgets.Button(description="Ask Question")
chat_output = widgets.Output()

display(question_input, ask_button, chat_output)

def on_ask_button_clicked(b):
    with chat_output:
        clear_output()
        try:
            print("Q: " + question_input.value)
            print("\nThinking...")
            response = chatbot.get_response(question_input.value)
            print("\nA: " + response)
            
            # Add feedback buttons
            display(HTML(
                "<div style='margin-top: 20px;'>Was this response helpful?</div>"
                "<button id='helpful-yes'>👍 Yes</button> "
                "<button id='helpful-no'>👎 No</button>"
            ))
        except Exception as e:
            print(f"Error: {str(e)}")
            print("\nPlease make sure you've initialized the chatbot in the section above.")

ask_button.on_click(on_ask_button_clicked)

## 2. Ad Copy Generation

Generate marketing ad copy for specific products and audiences.

In [4]:
product_input = widgets.Text(
    value='Cloud Marketing Platform',
    placeholder='Product/service name',
    description='Product:',
    disabled=False
)

audience_input = widgets.Text(
    value='Marketing directors at mid-size companies',
    placeholder='Target audience',
    description='Audience:',
    disabled=False
)

benefits_input = widgets.Textarea(
    value='Automated workflows, AI-powered insights, Performance tracking',
    placeholder='Key benefits (comma separated)',
    description='Benefits:',
    disabled=False,
    layout=widgets.Layout(width='500px')
)

generate_button = widgets.Button(description="Generate Ad Copy")
ad_output = widgets.Output()

display(product_input, audience_input, benefits_input, generate_button, ad_output)

def on_generate_button_clicked(b):
    with ad_output:
        clear_output()
        try:
            print("Generating ad copy...")
            benefits_list = [b.strip() for b in benefits_input.value.split(',')]
            ad_copy = chatbot.generate_ad_copy(
                product_input.value,
                audience_input.value,
                benefits_list
            )
            print("\nGenerated Ad Copy:\n")
            print(ad_copy)
            
            # Add rating stars
            display(HTML(
                "<div style='margin-top: 20px;'>Rate this ad copy (1-5 stars):</div>"
                "<div style='font-size: 24px;'>"
                "<span id='star1'>⭐</span>"
                "<span id='star2'>⭐</span>"
                "<span id='star3'>⭐</span>"
                "<span id='star4'>⭐</span>"
                "<span id='star5'>⭐</span>"
                "</div>"
            ))
        except Exception as e:
            print(f"Error: {str(e)}")
            print("\nPlease make sure you've initialized the chatbot in the section above.")

generate_button.on_click(on_generate_button_clicked)

## 3. A/B Test Variant Generation

Generate multiple variants of ad copy for A/B testing.

In [5]:
ab_product_input = widgets.Text(
    value='Social Media Management Tool',
    placeholder='Product/service name',
    description='Product:',
    disabled=False
)

ab_audience_input = widgets.Text(
    value='Small business marketers',
    placeholder='Target audience',
    description='Audience:',
    disabled=False
)

key_message_input = widgets.Text(
    value='Save time while increasing engagement',
    placeholder='Key message to convey',
    description='Key Message:',
    disabled=False,
    style={'description_width': 'initial'}
)

num_variants = widgets.IntSlider(
    value=3,
    min=2,
    max=5,
    step=1,
    description='Variants:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

ab_generate_button = widgets.Button(description="Generate A/B Variants")
ab_output = widgets.Output()

display(ab_product_input, ab_audience_input, key_message_input, num_variants, ab_generate_button, ab_output)

def on_ab_generate_button_clicked(b):
    with ab_output:
        clear_output()
        try:
            print(f"Generating {num_variants.value} variants...")
            variants = chatbot.generate_ab_test_variants(
                ab_product_input.value,
                ab_audience_input.value,
                key_message_input.value,
                num_variants=num_variants.value
            )
            print("\nGenerated A/B Test Variants:\n")
            for i, variant in enumerate(variants, 1):
                print(f"Variant {i}: {variant}\n")
                
            # Variant comparison
            display(HTML(
                "<div style='margin-top: 20px;'>Which variant do you prefer?</div>"
                "<select id='variant-preference'>"
                + "".join([f"<option value='{i}'>Variant {i}</option>" for i in range(1, len(variants)+1)]) +
                "</select> "
                "<button id='submit-preference'>Submit Preference</button>"
            ))
        except Exception as e:
            print(f"Error: {str(e)}")
            print("\nPlease make sure you've initialized the chatbot in the section above.")

ab_generate_button.on_click(on_ab_generate_button_clicked)

## 4. Model Performance Benchmarking

Compare the performance of different model configurations.

In [6]:
from quick_wins.benchmarking.performance_analyzer import PerformanceAnalyzer

benchmark_button = widgets.Button(description="Run Quick Benchmark")
benchmark_output = widgets.Output()

display(benchmark_button, benchmark_output)

def on_benchmark_button_clicked(b):
    with benchmark_output:
        clear_output()
        try:
            print("Running quick benchmark (this may take a few minutes)...")
            analyzer = PerformanceAnalyzer()
            
            # Benchmark just the current model with both quantized and unquantized
            analyzer.benchmark_model(model_dropdown.value, quantized=True, iterations=2, warmup=1)
            analyzer.benchmark_model(model_dropdown.value, quantized=False, iterations=2, warmup=1)
            
            # Get results
            df = analyzer.get_comparison_table()
            display(df)
            
            # Plot comparison
            analyzer.generate_charts("notebook_benchmark_results")
            
            # Show the most important chart
            plt.figure(figsize=(10, 6))
            df_plot = df.copy()
            df_plot['quantized'] = df_plot['quantized'].map({True: 'Quantized', False: 'Unquantized'})
            sns.barplot(x='quantized', y='avg_inference_sec', data=df_plot, palette="viridis")
            plt.title(f"Inference Time: {model_dropdown.value}")
            plt.ylabel("Average Inference Time (s)")
            plt.xlabel("")
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"Error: {str(e)}")
            import traceback
            traceback.print_exc()
            print("\nPlease make sure you've initialized the chatbot in the section above.")

benchmark_button.on_click(on_benchmark_button_clicked)